In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset


In [7]:
!python -c "import numpy; print(numpy.__version__)"

1.26.4


In [2]:
import sys
project_root = '../../src/'
sys.path.insert(0, project_root)

In [6]:
user_data = pd.read_csv('../../datasets/user1_1month_listening_history.csv')
print(user_data.head())

   duration (ms)  danceability  energy  loudness  speechiness  acousticness  \
0       125036.0      0.682982  0.4480  0.454525     0.582207      0.844378   
1       251480.0      0.884331  0.5360  0.580489     0.582207      0.048293   
2       129962.0      0.832923  0.5750  0.392388     0.582207      0.018976   
3       219333.0      0.358466  0.0525  0.467604     0.735099      0.897590   
4       175733.0      0.716183  0.2920  0.535565     0.582207      0.292169   

   instrumentalness  liveness  valence     tempo     spec_rate  labels  \
0          0.000000  0.188416    0.579  0.825227  4.754654e-07     0.0   
1          0.000135  0.179196    0.744  0.601342  7.754096e-07     1.0   
2          0.004970  0.447321    0.621  0.664681  4.754654e-07     2.0   
3          0.177000  0.447321    0.036  0.561521  5.060798e-07     0.0   
4          0.000748  0.546099    0.524  0.293744  4.754654e-07     0.0   

                                    uri  user_id  group_no  day  
0  spotify:tra

In [31]:
import numpy as np
import pandas as pd


def preprocess_data(df):
    """
    Preprocess the listening history to normalize features and prepare input for the RNN.
    """
    # Define feature columns
    feature_columns = [
        'duration (ms)', 'danceability', 'energy', 'loudness', 
        'speechiness', 'acousticness', 'instrumentalness', 
        'liveness', 'valence', 'tempo', 'spec_rate'
    ]
    
    # Normalize features
    scaler = MinMaxScaler()
    df[feature_columns] = scaler.fit_transform(df[feature_columns])
    
    # Convert the dataset into sequences for the RNN
    sequences = df[feature_columns].values
    
    return sequences, scaler


In [32]:
sequences

array([[0.22353366, 0.70700924, 0.44972848, ..., 0.58308157, 0.83963773,
        0.54294566],
       [0.57726868, 0.9188641 , 0.53809325, ..., 0.74924471, 0.61092788,
        0.88545934],
       [0.23731445, 0.8647735 , 0.57725491, ..., 0.62537764, 0.67563166,
        0.54294566],
       ...,
       [0.27798542, 0.69799414, 0.79615854, ..., 0.73615307, 0.71041177,
        0.32242579],
       [0.52415998, 0.7284201 , 0.60938755, ..., 0.62739174, 0.46904236,
        0.09578923],
       [0.45907445, 0.41852603, 0.14446837, ..., 0.5367573 , 0.35404596,
        0.25869604]])

In [52]:

import torch
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.5):
        super(RNNModel, self).__init__()
        
        # Define LSTM with multiple layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers=num_layers, 
                           batch_first=True, dropout=dropout)
        
        # Layer normalization for stability
        self.layer_norm = nn.LayerNorm(hidden_size)
        
        # Fully connected layers for projection
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, output_size)
        
        # Activation functions
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        
        # Dropout for regularization
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # Pass input through LSTM layers
        _, (hidden, _) = self.rnn(x)
        
        # Take the last hidden state of the last LSTM layer
        hidden = hidden[-1]
        
        # Normalize the hidden state
        hidden = self.layer_norm(hidden)
        
        # Pass through fully connected layers with activation
        out = self.fc1(hidden)
        out = self.relu(out)
        out = self.dropout(out)
        
        # Final projection to taste vector
        taste_vector = self.fc2(out)
        taste_vector = self.tanh(taste_vector)  # Optional, for bounded output
        
        return taste_vector

In [53]:
def train_rnn_model(model, train_loader, epochs=1000, learning_rate=0.001):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for sequences in train_loader:
            sequences = sequences.float()
            
            # Forward pass
            outputs = model(sequences)
            loss = criterion(outputs, sequences[:, -1])  # Predict the last song's vector
            epoch_loss += loss.item()
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_loader):.4f}")

In [54]:
from annoy import AnnoyIndex

def build_annoy_index(song_vectors, num_trees=10):
    """
    Build an Annoy index for nearest neighbor search.
    """
    num_features = song_vectors.shape[1]
    annoy_index = AnnoyIndex(num_features, 'euclidean')
    
    for i, vector in enumerate(song_vectors):
        annoy_index.add_item(i, vector)
    
    annoy_index.build(num_trees)
    return annoy_index

In [55]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up Spotify API credentials
client_id = '75d0ab19dcdc4db7821a27bf07df72a0'  # Replace with your Spotify client ID
client_secret = 'f64897e446834d7cb83b1c90916242df'  # Replace with your Spotify client secret

# Authenticate with Spotify
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to extract song name from Spotify URL
def get_song_names_from_url(song_urls):
    song_names = []
    for i in range(len(song_urls)):
        track_id = song_urls[i].split("/")[-1].split("?")[0]  # Extract the track ID from the URL
        track_info = sp.track(track_id)  # Get track information
        song_name = track_info['name']  # Extract song name
        artist_name = track_info['artists'][0]['name']  # Extract artist name
        song_names.append(f"{song_name} by {artist_name}")
    return song_names


In [56]:
def generate_recommendations(taste_vector, annoy_index, song_metadata, k):
    """
    Generate song recommendations by querying the Annoy index.
    """
    # Get nearest song indices
    nearest_indices = annoy_index.get_nns_by_vector(taste_vector, k, include_distances=False)
    print(nearest_indices)
    # Index into the song_metadata list directly
    recommended_songs = [song_metadata[i] for i in nearest_indices]
    return recommended_songs

In [60]:
if __name__ == "__main__":
    # Load dataset
    df =  pd.read_csv('../../datasets/user1_1month_listening_history.csv')  # Replace with your dataset path
    
    # Preprocess data
    sequences, scaler = preprocess_data(df)
    
    # Prepare DataLoader
    sequence_tensor = torch.tensor(sequences).unsqueeze(0)  # Add batch dimension
    train_loader = torch.utils.data.DataLoader(sequence_tensor, batch_size=1)
    
    # Define and train the RNN model
    input_size = sequences.shape[1]  # Number of features
    hidden_size = 128  # Size of the hidden layer
    output_size = sequences.shape[1]  # Output is the same size as input
    model = RNNModel(input_size, hidden_size, output_size)
    train_rnn_model(model, train_loader, epochs=250, learning_rate=0.01)
    
    # Generate the taste vector for the user
    model.eval()
    with torch.no_grad():
        taste_vector = model(sequence_tensor.float()).squeeze(0).numpy()
    
    # Build the Annoy index from the entire dataset
    song_vectors = sequences  # Use normalized song features
    annoy_index = build_annoy_index(song_vectors)
    
    # Generate recommendations
  # Generate recommendations
  # Generate recommendations
    song_metadata = df['uri'].tolist()  # Convert 'uri' column to a list
    recommended_songs_uris = generate_recommendations(taste_vector, annoy_index, song_metadata, k=5)

    # Fetch song names using the Spotify API
    recommended_songs = get_song_names_from_url(recommended_songs_uris)

    # Display the recommended songs
    print("\nRecommended Songs:")
    for song in recommended_songs:
        print(song)

Epoch 1, Loss: 0.2346
Epoch 2, Loss: 0.1830
Epoch 3, Loss: 0.1301
Epoch 4, Loss: 0.1694
Epoch 5, Loss: 0.3232
Epoch 6, Loss: 0.1212
Epoch 7, Loss: 0.0717
Epoch 8, Loss: 0.0932
Epoch 9, Loss: 0.1097
Epoch 10, Loss: 0.0750
Epoch 11, Loss: 0.1397
Epoch 12, Loss: 0.0659
Epoch 13, Loss: 0.1434
Epoch 14, Loss: 0.3093
Epoch 15, Loss: 0.2002
Epoch 16, Loss: 0.1387
Epoch 17, Loss: 0.0820
Epoch 18, Loss: 0.0472
Epoch 19, Loss: 0.0510
Epoch 20, Loss: 0.0539
Epoch 21, Loss: 0.1139
Epoch 22, Loss: 0.0624
Epoch 23, Loss: 0.1407
Epoch 24, Loss: 0.0251
Epoch 25, Loss: 0.0945
Epoch 26, Loss: 0.0745
Epoch 27, Loss: 0.0782
Epoch 28, Loss: 0.0519
Epoch 29, Loss: 0.0526
Epoch 30, Loss: 0.1011
Epoch 31, Loss: 0.0146
Epoch 32, Loss: 0.0370
Epoch 33, Loss: 0.0629
Epoch 34, Loss: 0.0243
Epoch 35, Loss: 0.0451
Epoch 36, Loss: 0.0102
Epoch 37, Loss: 0.0419
Epoch 38, Loss: 0.0153
Epoch 39, Loss: 0.0166
Epoch 40, Loss: 0.0080
Epoch 41, Loss: 0.0332
Epoch 42, Loss: 0.0146
Epoch 43, Loss: 0.0040
Epoch 44, Loss: 0.00

In [45]:
print(nearest_indices) 

NameError: name 'nearest_indices' is not defined

In [44]:
print(taste_vector)  

[0.47073826 0.4445556  0.12904939 0.45111683 0.32960016 0.8910057
 0.00118578 0.3523093  0.5384249  0.36608356 0.30806583]


In [44]:
if __name__ == "__main__":
    # Load dataset
    df = pd.read_csv('../../datasets/seven_day_listening_history.csv')
    
    # Preprocess data
    sequences, targets, uris = preprocess_data(df)
    
    # Prepare DataLoader
    train_data = TensorDataset(torch.tensor(sequences), torch.tensor(targets))
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    
    # Define model
    input_size = sequences.shape[2]
    hidden_size = 128
    output_size = targets.shape[1]
    model = LSTMModel(input_size, hidden_size, output_size)
    
    # Train model
    train_lstm_model(model, train_loader, epochs=1000, learning_rate=0.0001)
    
    # Generate predictions
    model.eval()
    predicted_features = model(torch.tensor(sequences).float()).detach()
    
    # Use cosine similarity to recommend songs
    all_song_features = df[[
        'danceability', 'energy', 'loudness', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'duration (ms)', 'spec_rate'
    ]].values
    song_uris = df['uri'].values
    recommended_songs = get_recommendations(predicted_features, all_song_features, song_uris)
    
    # Print recommendations
    for user_idx, recs in enumerate(recommended_songs):
        print(f"Recommendations for User {user_idx + 1}: {recs}")

Epoch 1, Loss: 4865558592.0000
Epoch 2, Loss: 4460408064.0000
Epoch 3, Loss: 4246406720.0000
Epoch 4, Loss: 4166530112.0000
Epoch 5, Loss: 4418362304.0000
Epoch 6, Loss: 3915305664.0000
Epoch 7, Loss: 4236339328.0000
Epoch 8, Loss: 4404446400.0000
Epoch 9, Loss: 4549164352.0000
Epoch 10, Loss: 4223880128.0000
Epoch 11, Loss: 4357630656.0000
Epoch 12, Loss: 4244770176.0000
Epoch 13, Loss: 3970239488.0000
Epoch 14, Loss: 4255937664.0000
Epoch 15, Loss: 4658964672.0000
Epoch 16, Loss: 5049889728.0000
Epoch 17, Loss: 4454228224.0000
Epoch 18, Loss: 4076239168.0000
Epoch 19, Loss: 4036353408.0000
Epoch 20, Loss: 4108065280.0000
Epoch 21, Loss: 4380362624.0000
Epoch 22, Loss: 4343662720.0000
Epoch 23, Loss: 4517248448.0000
Epoch 24, Loss: 4422633984.0000
Epoch 25, Loss: 4176136000.0000
Epoch 26, Loss: 4428577280.0000
Epoch 27, Loss: 3999497600.0000
Epoch 28, Loss: 4750724672.0000
Epoch 29, Loss: 4372513344.0000
Epoch 30, Loss: 4320428096.0000
Epoch 31, Loss: 4218984192.0000
Epoch 32, Loss: 4

/var/folders/x6/74dnvpqn4wn204zbb5m88mzw0000gn/T/ipykernel_76624/3365809677.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_feat_tensor = torch.tensor(pred_feat).float().unsqueeze(0)


Recommendations for User 1: ['The Boondocks Main Title by Asheru', 'Elevator Jam by LSPLASH', 'Little Toy Guns- Story Behind the Song by Carrie Underwood', 'El Mal Villano by Raíces', 'no lyrics needed pt. viii by no lyrics']
Recommendations for User 2: ['The Boondocks Main Title by Asheru', 'Elevator Jam by LSPLASH', 'Little Toy Guns- Story Behind the Song by Carrie Underwood', 'El Mal Villano by Raíces', 'no lyrics needed pt. viii by no lyrics']
Recommendations for User 3: ['The Boondocks Main Title by Asheru', 'Elevator Jam by LSPLASH', 'Little Toy Guns- Story Behind the Song by Carrie Underwood', 'El Mal Villano by Raíces', 'no lyrics needed pt. viii by no lyrics']
Recommendations for User 4: ['The Boondocks Main Title by Asheru', 'Elevator Jam by LSPLASH', 'Little Toy Guns- Story Behind the Song by Carrie Underwood', 'El Mal Villano by Raíces', 'no lyrics needed pt. viii by no lyrics']
Recommendations for User 5: ['The Boondocks Main Title by Asheru', 'Elevator Jam by LSPLASH', 'L

In [32]:
print("Sample Input Sequences:", sequences[:2])  # Check if sequences differ for users

Sample Input Sequences: [[[6.82981686e-01 4.48000000e-01 4.54525435e-01 5.82207102e-01
   8.44377510e-01 0.00000000e+00 1.88416076e-01 5.79000000e-01
   8.25226517e-01 1.25036000e+05 4.75465388e-07]
  [5.12691443e-01 2.06000000e-01 2.55633855e-01 3.38410596e-01
   9.87951807e-01 9.27000000e-01 2.57683215e-01 3.30000000e-01
   3.23827274e-01 1.54280000e+05 3.31215971e-07]
  [6.78697654e-01 8.07000000e-01 6.10181311e-01 2.57615894e-01
   1.04417671e-01 2.19000000e-01 2.74231678e-01 3.82000000e-01
   4.86122262e-01 3.32587000e+05 1.16961878e-07]
  [5.82306951e-01 8.98000000e-01 7.11517287e-01 2.86092715e-01
   8.13253012e-02 0.00000000e+00 2.81323877e-01 7.07000000e-01
   6.01020759e-01 2.37373000e+05 1.81992055e-07]
  [4.65567099e-01 8.79000000e-01 6.66593261e-01 7.88079470e-01
   1.18473896e-03 8.20000000e-06 1.00000000e+00 3.95000000e-01
   6.13797454e-01 2.88805000e+05 4.12042728e-07]
  [8.41490843e-01 5.59000000e-01 4.28613374e-01 5.82207102e-01
   1.32530120e-01 0.00000000e+00 1.728

In [33]:
print("Predicted Features for First User:", predicted_features[0])
print("Predicted Features for Second User:", predicted_features[1])

Predicted Features for First User: tensor([0.5437, 0.6280, 0.5374, 0.4440, 0.3576, 0.1955, 0.3806, 0.4961, 0.5760,
        1.2153, 0.0117])
Predicted Features for Second User: tensor([0.5437, 0.6280, 0.5374, 0.4440, 0.3576, 0.1955, 0.3806, 0.4961, 0.5760,
        1.2153, 0.0117])
